In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import panel as pn

pn.extension()

The ``Tabulator`` widget allows displaying and editing a pandas DataFrame. The `Tabulator` is a largely backward compatible replacement for the [`DataFrame`](./DataFrame.ipynb) widget and will eventually replace it. It is built on the [Tabulator](http://tabulator.info/) library, which provides for a wide range of features.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``editors``** (``dict``):  A dictionary mapping from column name to a bokeh `CellEditor` instance or tabulator editor specification.
* **``formatters``** (``dict``): A dictionary mapping from column name to a bokeh `CellFormatter` instance or tabulator formatter specification.
* **``layout``** (str): Describes the column layout mode with one of the following options `'fit_columns'`, `'fit_data'`, `'fit_data_stretch'`, `'fit_data_fill'`, `'fit_data_table'`. 
* **``frozen_columns``** (`list`): List of columns to freeze, preventing them from scrolling out of frame. Column can be specified by name or index.
* **``frozen_rows``**: (`list`): List of rows to freeze preventing them from scrolling out of frame. Rows can be specified by positive or negative index.
* **``page``** (``int``): Current page if pagination is enabled.
* **``page_size``** (``int``): Number of rows on each page.
* **``pagination``** (`str`, `default=None`):  Set to 'local' or 'remote' to enable pagination, by default pagination is disabled with the value set to `None`.
* **``selection``** (``list``): The currently selected rows.
* **``selectable``** (`boolean` or `str`): Whether to allow selection of rows. Can be `True`, `False` or `'checkbox'. 
* **``show_index``** (``boolean``): Whether to show the index column.
* **`titles`** (``dict``): A mapping from column name to a title to override the name with.
* **``value``** (``pd.DataFrame``): The pandas DataFrame to display and edit
* **``widths``** (``dict``): A dictionary mapping from column name to column width in the rendered table.

##### Display

* **``disabled``** (boolean): Whether the widget is editable
* **``name``** (str): The title of the widget

___

The ``Tabulator`` widget renders a DataFrame using an interactive grid, which allows directly editing the contents of the dataframe inplace, with any changes being synced with Python. The `Tabulator` will usually determine the appropriate formatter appropriately based on the type of the data:

In [ ]:
df = pd.DataFrame({
    'int': [1, 2, 3],
    'float': [3.14, 6.28, 9.42],
    'str': ['A', 'B', 'C'],
    'bool': [True, False, True],
    'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10)]
}, index=[1, 2, 3])

df_widget = pn.widgets.Tabulator(df)
df_widget

## Formatters

By default the widget will pick bokeh ``CellFormatter`` and ``CellEditor`` types appropriate to the dtype of the column. These may be overriden by explicit dictionaries mapping from the column name to the editor or formatter instance. For example below we create a ``SelectEditor`` instance to pick from four options in the ``str`` column and a ``NumberFormatter`` to customize the formatting of the float values:

In [ ]:
from bokeh.models.widgets.tables import NumberFormatter, BooleanFormatter

bokeh_formatters = {
    'float': NumberFormatter(format='0.00000'),
    'bool': BooleanFormatter(),
}

pn.widgets.Tabulator(df, formatters=bokeh_formatters)

The list of valid Bokeh formatters includes:
    
* [BooleanFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.BooleanFormatter)
* [DateFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.DateFormatter)
* [NumberFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.NumberFormatter)
* [HTMLTemplateFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.HTMLTemplateFormatter)
* [StringFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.StringFormatter)
* [ScientificFormatter](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.ScientificFormatter)

However in addition to the formatters exposed by Bokeh it is also possible to provide valid formatters built into the Tabulator library. These may be defined either as a string or as a dictionary declaring the 'type' and other arguments, which are passed to Tabulator as the `formatterParams`:

In [ ]:
tabulator_formatters = {
    'int': 'start',
    'float': {'type': 'progress', 'max': 10},
    'bool': {'type': 'tickCross'}
}

pn.widgets.Tabulator(df, formatters=tabulator_formatters)

The list of valid Tabulator formatters can be found in the [Tabulator documentation](http://tabulator.info/docs/4.9/format#format-builtin).

## Editors

Just like the formatters the `Tabulator` will natively understand the Bokeh `Editor` types, however in the background it will replace most of them with equivalent editors natively supported by the tabulator library:

In [ ]:
from bokeh.models.widgets.tables import CheckboxEditor, NumberEditor, SelectEditor, DateEditor, TimeEditor
df['time'] = '00:00:00'

bokeh_editors = {
    'float': NumberEditor(),
    'bool': CheckboxEditor(),
    'str': SelectEditor(options=['A', 'B', 'C', 'D']),
}

pn.widgets.Tabulator(df[['float', 'bool', 'str', 'time']], editors=bokeh_editors)

Therefore it is often preferable to use one of the [Tabulator editors](http://tabulator.info/docs/4.9/edit#edit) directly:

In [ ]:
from bokeh.models.widgets.tables import CheckboxEditor, NumberEditor, SelectEditor

bokeh_editors = {
    'float': {'type': 'number', 'max': 10, 'step': 0.1},
    'bool': {'type': 'tickCross', 'tristate': True, 'indeterminateValue': None},
    'str': {'type': 'autocomplete', 'values': True}
}

pn.widgets.Tabulator(df[['float', 'bool', 'str']], editors=bokeh_editors)

### Column layouts

By default the DataFrame widget will adjust the sizes of both the columns and the table based on the contents, reflecting the default value of the parameter: `layout="fit_data_table"`. Alternatively modes allow manually specifying the widths of the columns, giving each column equal widths or adjusting just the size of the columns.

#### Manual column widths

To manually adjust column widths provide explicit `widths` for each of the columns:

In [ ]:
custom_df = pd.util.testing.makeMixedDataFrame()

pn.widgets.Tabulator(custom_df, widths={'index': 70, 'A': 50, 'B': 50, 'C': 70, 'D': 130})

You can also declare a single width for all columns this way:

In [ ]:
pn.widgets.Tabulator(custom_df, widths=130)

#### Autosize columns

To automatically adjust the columns dependending on their content set `layout='fit_data'`:

In [ ]:
pn.widgets.tables.Tabulator(custom_df, layout='fit_data', width=400)

To ensure that the table fits all the data but also stretches to fill all the available space set `layout='fit_data_stretch'`:

In [ ]:
pn.widgets.tables.Tabulator(custom_df, layout='fit_data_stretch', width=400)

The `'fit_data_fill'` option on the other hand won't stretch the last column but still fill the space:

In [ ]:
pn.widgets.tables.Tabulator(custom_df, layout='fit_data_fill', width=400)

Perhaps the most useful of these options is `layout='fit_data_table'` (and therefore the default) since this will automatically size both the columns and the table:

In [ ]:
pn.widgets.tables.Tabulator(custom_df, layout='fit_data_table')

#### Equal size

The simplest option is simply to allocate each column equal amount of size:

In [ ]:
pn.widgets.tables.Tabulator(custom_df, layout='fit_columns', width=650)

## Styling

The ability to style the contents of a table based on its content and other considerations is very important. Thankfully pandas provides a powerful [styling API](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html), which can be used in conjunction with the `Tabulator` widget. Specifically the `Tabulator` widget exposes a `.style` attribute just like a `pandas.DataFrame` which lets the user apply custom styling using methods like `.apply` and `.applymap`. For a detailed guide to styling see the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html).

Here we will demonstrate with a simple example, starting with a basic table:

In [ ]:
style_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))
styled = pn.widgets.Tabulator(style_df, page_size=5)

Next we define two functions which apply styling cell-wise (`color_negative_red`) and column-wise (`highlight_max`), which we then apply to the `Tabulator` using the `.style` API and then display the `styled` table:

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

styled.style.applymap(color_negative_red).apply(highlight_max)

styled

## Selection

The `selection` parameter controls which rows in the table are selected and can be set from Python and updated by selecting rows on the frontend:

In [ ]:
sel_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))

select_table = pn.widgets.tables.Tabulator(sel_df, selection=[0, 3, 7])
select_table

Once initialized the ``selection`` parameter will return the integer indexes of the selected rows and the ``selected_dataframe`` property will return a new DataFrame containing just the selected rows:

In [ ]:
select_table.selection = [1, 4, 9]

select_table.selected_dataframe

The `selectable` parameter can also be used to control how the selection works, setting it to `False` will disable selecting entirely, while setting `selectable='checkbox'` will add checkboxes to perform the selection:

In [ ]:
pn.widgets.tables.Tabulator(sel_df, selection=[0, 3, 7], selectable='checkbox')

### Freezing rows and columns

Often times your table will be larger than can be displayed in a single viewport and scroll bars will be enabled. The issue with this is that you might want to make sure that certain information is always visible. This is where the `frozen_columns` and `frozen_rows` options come in.

#### Frozen columns

When you have a large number of columns and can't fit them all on the screen you might still want to make sure that certain columns do not scroll out of view. The `frozen_columns` option makes this possible by specifying a list of columns that should be frozen, e.g. `frozen_columns=['index']` will freeze the index column:

In [ ]:
wide_df = pd.util.testing.makeCustomDataframe(10, 10, r_idx_names=['index'])

pn.widgets.Tabulator(wide_df, frozen_columns=['index'], width=400)

#### Frozen rows

Another common scenario is when you have certain rows with special meaning, e.g. aggregates which summarize the information in the rest of the table. In this case you may want to freeze those rows so they do not scroll out of view. You can achieve this by setting a list of `frozen_rows` by integer index (which can be positive or negative):

In [ ]:
date_df = pd.util.testing.makeTimeDataFrame().iloc[:10]
agg_df = pd.concat([date_df, date_df.median().to_frame('Median').T, date_df.mean().to_frame('Mean').T])
agg_df.index= agg_df.index.map(str)

pn.widgets.tables.Tabulator(agg_df, frozen_rows=[-2, -1], width=400)

## Grouping

Another useful option is the ability to group specific rows together, which can be achieved using `groups` parameter. The `groups` parameter should be composed of a dictionary mapping from the group titles to the column names:

In [ ]:
pn.widgets.tables.Tabulator(date_df, width=400, groups={'Group 1': ['A', 'B'], 'Group 2': ['C', 'D']})

## Pagination

When working with large tables we sometimes can't send all the data to the browser at once. In these scenarios we can enable pagination, which will fetch only the currently viewed data from the server backend. This may be enabled by setting `pagination='remote'` and the size of each page can be set using the `page_size` option:

In [ ]:
large_df = pd.util.testing.makeCustomDataframe(100000, 5) 

paginated_table = pn.widgets.tables.Tabulator(large_df, pagination='remote', page_size=10)
paginated_table

## Filtering

A very common scenario is that you want to attach a number of filters to a table in order to view just a subset of the data. You can achieve this through callbacks or other reactive approaches but the  `.add_filter` method makes it much easier.

#### Widget filters

The simplest approach to filtering is simply to declare one or more widgets and then using the `.add_filter` to add them as filters on specific columns. The filtering will depend on the type of the widget or parameters value:

- scalar: Filters by checking for equality
- tuple: A tuple will be interpreted as range.
- list: A list will be interpreted as a set of discrete scalars and the filter will check if the values in the column match any of the items in the list.

In [ ]:
mixed_df = pd.util.testing.makeMixedDataFrame()

filter_table = pn.widgets.Tabulator(mixed_df)

slider = pn.widgets.RangeSlider(start=0, end=4, name='A Filter')
select = pn.widgets.MultiSelect(options=['foo1', 'foo2', 'foo3', 'foo4', 'foo5'], name='C Filter')

filter_table.add_filter(slider, 'A')
filter_table.add_filter(select, 'C')

pn.Row(
    pn.Column(slider, select),
    filter_table
)

For more complex filtering tasks you can supply a function, which should accept the DataFrame to be filtered as the first argued and must return a filtered copy of the data. By using the `pn.bind` function, which binds widget and parameter values to a function, complex filtering can be achieved, e.g. here we will add a filter function which uses `fnmatch` (similar to a regex) to filter on the 'Director' column of a listing of thousands of movies:

In [ ]:
import fnmatch
import sqlite3

from bokeh.sampledata.movies_data import movie_path

con = sqlite3.Connection(movie_path)

movies_df = pd.read_sql('SELECT Title, Year, Genre, Director, Writer, imdbRating from omdb', con)

director_filter = pn.widgets.TextInput(name='Director filter', value='*Chaplin')

def filter_fnmatch(df, pattern, column):
    if not pattern:
        return df
    mask = [fnmatch.fnmatch(v, pattern) if v else False for v in df[column]]
    return df[mask]

movies_table = pn.widgets.Tabulator(movies_df, pagination='remote', layout='fit_columns', width=800)
    
movies_table.add_filter(pn.bind(filter_fnmatch, pattern=director_filter, column='Director'))    

pn.Row(director_filter, movies_table)

## Streaming

When we are monitoring some source of data which updates we may want to update the table with the newly arriving data, however we do not want to transmit the entire dataset each time. To handle efficient transfer of just the latest data we can use the `.stream` method on the `Tabulator` object:

In [ ]:
stream_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))

stream_table = pn.widgets.Tabulator(stream_df, layout='fit_columns', width=450)
stream_table

As example we will schedule a periodic callback which will stream new data every 1000ms (i.e. 1s) five times in a row:

In [ ]:
def stream_data(follow=True):
    stream_df = pd.DataFrame(np.random.randn(10, 5), columns=list('ABCDE'))
    stream_table.stream(stream_df, follow=follow)

pn.state.add_periodic_callback(stream_data, period=1000, count=5)

If you are viewing this example in a live kernel you will be able to watch the table update and scroll along. If we want to disable the scrolling behavior we can set `follow=False`:

In [ ]:
stream_data(follow=False)

## Patching

In certain cases we don't want to update the table with new data but just patch existing data.

In [ ]:
patch_table = pn.widgets.Tabulator(df[['int', 'float', 'str', 'bool']])
patch_table

The easiest way to patch the data is by supplying a dictionary as the patch value. The dictionary should have the following structure:

```python
{
    column: [
        (index: int or slice, value),
        ...
    ],
    ...
}
```
    
As an example, below we will patch the 'bool' and 'int' columns. On the `'bool'` column we will replace the 0th and 2nd row and on the `'int'` column we replace the first two rows:

In [ ]:
patch_table.patch({
    'bool': [
        (0, False),
        (2, False)
    ],
    'int': [
        (slice(0, 2), [3, 2])
    ]
})